In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [50]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V = V.reshape((-1,1))
    
    R_pi = np.array([np.sum([policy[s][a] * env.P[s][a][0][2]
                             for a in range(env.nA)])
                     for s in range(env.nS)])
    R_pi = R_pi.reshape((-1,1))

    P_pi = np.array([
                        [
                             np.sum([policy[curr_state][a]
                             for a in range(env.nA)
                             if env.P[curr_state][a][0][1]==next_state])
                             
                         for next_state in range(env.nS)]
                     for curr_state in range(env.nS)])

    while True:
        # TODO: Implement!
        V_new = R_pi + discount_factor * np.matmul(P_pi, V)
        if np.max(np.abs(V_new - V)) < theta:
            V = V_new
            break
        V = V_new
    return V.flatten()

In [ ]:
[policy[curr_state][a]
 for a in range(env.nA)
 if env.P[curr_state][a][0][1]==next_state]

prob, next_state, reward, _for env.P[curr_state][a]

In [55]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    q_pi = np.zeros((env.nS, env.nA),
                        dtype=np.float64)
    min_diff = 0.01
    
    while True:
        # Implement this!
        V = policy_eval_fn(policy=policy, 
                           env=env, 
                           discount_factor=discount_factor,
                           theta=0.00001)
        q_pi.fill(0.)
        for curr_state in range(env.nS):
            for a in range(env.nA):
                prob, next_state, reward, _ = env.P[curr_state][a][0]
                q_pi[curr_state][a] = reward + discount_factor * prob * V[next_state]
                
        # Bellman optimality equation
        if np.max(np.abs(np.max(q_pi, axis=1) - V)) < min_diff:
            break
        
        best_act = np.argmax(q_pi, axis=1)
        policy.fill(0.)
        for s in range(env.nS):
            policy[s][best_act[s]] = 1.
        
    return policy, V

In [56]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 3 2 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [57]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [58]:
import timeit
n = 100
t = timeit.timeit("a = policy_improvement(env)", globals=globals(), number=n) / n
print("avg time: {}".format(t))

avg time: 0.006184362049971241
